## Imports

In [ ]:
# imports
import pandas as pd
import re
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, Dropout
from tensorflow.keras.layers import LSTM
import numpy as np
import pickle
import keras
from tensorflow.keras.constraints import unit_norm
from tensorflow.keras.callbacks import EarlyStopping
'''
from helper_functions import load_data
from helper_functions import my_split
from helper_functions import upsample_minority
from helper_functions import downsample_majority
from helper_functions import model_prep
from helper_functions import get_results
from helper_functions import get_f1
from helper_functions import clean_text
'''

'\nfrom helper_functions import load_data\nfrom helper_functions import my_split\nfrom helper_functions import upsample_minority\nfrom helper_functions import downsample_majority\nfrom helper_functions import model_prep\nfrom helper_functions import get_results\nfrom helper_functions import get_f1\nfrom helper_functions import clean_text\n'

In [ ]:
!pip install category_encoders

     |████████████████████████████████| 81kB 2.1MB/s 


In [ ]:
# imports
import os
import pandas as pd
from sklearn.utils import resample
import category_encoders as ce
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import f1_score
import keras.backend as K
import re


def load_data():
    FILE_PATH = os.path.join(os.getcwd(), 'data', 'large_data.csv')
    return pd.read_csv(FILE_PATH, index_col=None)

def upsample_minority(df):
    counts = df['final_status'].value_counts().index
    majority = counts[0]
    minority = counts[1]

    df_majority = df[df['final_status'] == majority]
    df_minority = df[df['final_status'] == minority]

    majority_class_size = len(df_majority)
    minority_class_size = len(df_minority)

    minority_upsampled = resample(df_minority, 
                replace=True, 
                n_samples=majority_class_size,
                random_state=42) 
    return pd.concat([df_majority, minority_upsampled])

def downsample_majority(df):
    counts = df['final_status'].value_counts().index
    majority = counts[0]
    minority = counts[1]

    df_majority = df[df['final_status'] == majority]
    df_minority = df[df['final_status'] == minority]

    majority_class_size = len(df_majority)
    minority_class_size = len(df_minority)

    majority_downsampled = resample(df_majority, 
                replace=False, 
                n_samples=minority_class_size,
                random_state=42) 
    return pd.concat([df_minority, majority_downsampled])

def my_split(df, year):
    train = df[df['launch_year'] < year]
    test = df[df['launch_year'] == year]
    return train, test

def model_prep(train, test, features, target, onehot=True, scale=True):
    encoder = ce.one_hot.OneHotEncoder(use_cat_names=True)
    scaler = StandardScaler()

    X_train = train[features]
    if onehot:
        X_train = encoder.fit_transform(X_train)
    if scale:
        X_train = scaler.fit_transform(X_train)

    y_train = train[target]

    X_test = test[features]
    if onehot:
        X_test = encoder.transform(X_test)
    if scale:
        X_test = scaler.transform(X_test)
    y_test = test[target]
    return X_train, y_train, X_test, y_test

def get_results(y_true, y_pred):
    accuracy_metric = accuracy_score(y_true, y_pred)
    roc_auc_metric = roc_auc_score(y_true, y_pred)
    f1_metric = f1_score(y_true, y_pred)

    print('-------------------------------')
    print(f'Accuracy Score: {accuracy_metric}')
    print(f'ROC AUC Score: {roc_auc_metric}')
    print(f'F1 Score: {f1_metric}')
    return

# code credit to https://medium.com/@aakashgoel12/how-to-add-user-defined-function-get-f1-score-in-keras-metrics-3013f979ce0d
def get_f1(y_true, y_pred): #taken from old keras source code
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    recall = true_positives / (possible_positives + K.epsilon())
    f1_val = 2*(precision*recall)/(precision+recall+K.epsilon())
    return f1_val

def clean_text(text):
    tokens = re.sub('[^a-zA-Z 0-9]', '', text)
    tokens = tokens.lower().split()
    return tokens


/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [ ]:
# loading the data
'''
df = load_data()
df.head()
'''
df = pd.read_csv('/content/large_data.csv')

In [ ]:
# setting variables
batch_size = 32
max_features = 10000
features = 'name'
target = 'final_status'
maxlen= 10
oov_token = max_features+1

In [ ]:
oov_token

10001

In [ ]:
# cleaning the data
df[features] = df[features].fillna('')
df[features] = df[features].apply(lambda x: clean_text(x))

In [ ]:
# train/test split
year = 2020
train, test = my_split(df, year)

# transforming words to integer values
tokenizer = Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(train[features])

train[features] = tokenizer.texts_to_sequences(train[features])
test[features] = tokenizer.texts_to_sequences(test[features])

# processing data
X_train, y_train, X_test, y_test = model_prep(train, test, features, target, onehot=False, scale=False)
#maxlen = max([len(each) for each in train[features]])

# padding sequences to all be the same length
X_train = sequence.pad_sequences(X_train, maxlen=maxlen, padding='post')
X_test = sequence.pad_sequences(X_test, maxlen=maxlen, padding='post')

# instantiating the model
model = Sequential()
model.add(Embedding(max_features, 128))
model.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(1, activation='sigmoid'))

# compiling the model
model.compile(loss='binary_crossentropy',
              optimizer='adam', 
              metrics=['accuracy', get_f1])

# fitting the model
history = model.fit(X_train, y_train,
          batch_size=batch_size, 
          epochs=5, 
          validation_data=(X_test,y_test))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.


Epoch 1/5
9896/9896 [==============================] - 805s 81ms/step - loss: 0.6025 - accuracy: 0.6526 - get_f1: 0.5908 - val_loss: 0.5331 - val_accuracy: 0.6966 - val_get_f1: 0.7393
Epoch 2/5
9896/9896 [==============================] - 796s 80ms/step - loss: 0.5759 - accuracy: 0.6802 - get_f1: 0.6281 - val_loss: 0.5376 - val_accuracy: 0.6927 - val_get_f1: 0.7360
Epoch 3/5
9896/9896 [==============================] - 802s 81ms/step - loss: 0.5578 - accuracy: 0.6945 - get_f1: 0.6463 - val_loss: 0.5206 - val_accuracy: 0.7167 - val_get_f1: 0.7666
Epoch 4/5
9896/9896 [==============================] - 804s 81ms/step - loss: 0.5393 - accuracy: 0.7104 - get_f1: 0.6643 - val_loss: 0.5379 - val_accuracy: 0.6969 - val_get_f1: 0.7442
Epoch 5/5
9896/9896 [==============================] - 797s 80ms/step - loss: 0.5189 - accuracy: 0.7261 - get_f1: 0.6836 - val_loss: 0.5759 - val_accuracy: 0.6829 - val_get_f1: 0.7259


In [ ]:
y_pred = (model.predict(X_test) > 0.5).astype("int32")
y_true = y_test

get_results(y_true, y_pred)

-------------------------------
Accuracy Score: 0.6829078992756122
ROC AUC Score: 0.7036824448399581
F1 Score: 0.7415840888326658


In [ ]:

with open('tokenizer.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
model.save('basic_model.h5')

In [ ]:
# train/test split
year = 2020
train, test = my_split(df, year)

# transforming words to integer values
tokenizer = Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(train[features])

train[features] = tokenizer.texts_to_sequences(train[features])
test[features] = tokenizer.texts_to_sequences(test[features])

# processing data
X_train, y_train, X_test, y_test = model_prep(train, test, features, target, onehot=False, scale=False)
#maxlen = max([len(each) for each in train[features]])

# padding sequences to all be the same length
X_train = sequence.pad_sequences(X_train, maxlen=maxlen, padding='post')
X_test = sequence.pad_sequences(X_test, maxlen=maxlen, padding='post')

# instantiating the model
model = Sequential()
model.add(Embedding(max_features, 128))
model.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(10, activation='relu'))
model.add(Dropout(.1))
model.add(Dense(1, activation='sigmoid'))

# compiling the model
model.compile(loss='binary_crossentropy',
              optimizer='adam', 
              metrics=['accuracy'])

# fitting the model
history = model.fit(X_train, y_train,
          batch_size=batch_size, 
          epochs=5, 
          validation_data=(X_test,y_test))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.


Epoch 1/5
9896/9896 [==============================] - 782s 79ms/step - loss: 0.6053 - accuracy: 0.6505 - val_loss: 0.5512 - val_accuracy: 0.6814
Epoch 2/5
9896/9896 [==============================] - 816s 82ms/step - loss: 0.5778 - accuracy: 0.6787 - val_loss: 0.5524 - val_accuracy: 0.6895
Epoch 3/5
9896/9896 [==============================] - 829s 84ms/step - loss: 0.5605 - accuracy: 0.6932 - val_loss: 0.5270 - val_accuracy: 0.7046
Epoch 4/5
9896/9896 [==============================] - 838s 85ms/step - loss: 0.5418 - accuracy: 0.7092 - val_loss: 0.5546 - val_accuracy: 0.6887
Epoch 5/5
9896/9896 [==============================] - 835s 84ms/step - loss: 0.5217 - accuracy: 0.7254 - val_loss: 0.5704 - val_accuracy: 0.6866


In [ ]:
model.save('basic_name_model.h5')

In [ ]:
y_pred = (model.predict(X_test) > 0.5).astype("int32")
y_true = y_test

get_results(y_true, y_pred)

-------------------------------
Accuracy Score: 0.6866160745084512
ROC AUC Score: 0.7047922140294375
F1 Score: 0.746051712089448


In [ ]:
# it appears that model was slightly better so training it with more epochs
# train/test split
year = 2020
train, test = my_split(df, year)

# transforming words to integer values
tokenizer = Tokenizer(num_words=max_features, oov_token=oov_token)
tokenizer.fit_on_texts(train[features])

train[features] = tokenizer.texts_to_sequences(train[features])
test[features] = tokenizer.texts_to_sequences(test[features])

# processing data
X_train, y_train, X_test, y_test = model_prep(train, test, features, target, onehot=False, scale=False)
#maxlen = max([len(each) for each in train[features]])

# padding sequences to all be the same length
X_train = sequence.pad_sequences(X_train, maxlen=maxlen, padding='post')
X_test = sequence.pad_sequences(X_test, maxlen=maxlen, padding='post')

# instantiating the model
model = Sequential()
model.add(Embedding(max_features, 128))
model.add(LSTM(128, 
               dropout=0.2, recurrent_dropout=0.3,
               kernel_constraint=unit_norm(), recurrent_constraint=unit_norm()))
model.add(Dense(1, activation='sigmoid'))

# compiling the model
model.compile(loss='binary_crossentropy',
              optimizer='adam', 
              metrics=['accuracy'])

es = EarlyStopping(monitor='val_loss', mode='min', patience=5)
# fitting the model
history = model.fit(X_train, y_train,
          batch_size=batch_size, 
          epochs=25, 
          validation_data=(X_test,y_test),
          callbacks=[es])

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()


Epoch 1/25
9896/9896 [==============================] - 515s 52ms/step - loss: 0.6013 - accuracy: 0.6548 - val_loss: 0.5348 - val_accuracy: 0.6989
Epoch 2/25
9896/9896 [==============================] - 518s 52ms/step - loss: 0.5748 - accuracy: 0.6815 - val_loss: 0.5580 - val_accuracy: 0.6651
Epoch 3/25
9896/9896 [==============================] - 528s 53ms/step - loss: 0.5585 - accuracy: 0.6953 - val_loss: 0.5444 - val_accuracy: 0.6988
Epoch 4/25
9896/9896 [==============================] - 522s 53ms/step - loss: 0.5447 - accuracy: 0.7066 - val_loss: 0.5374 - val_accuracy: 0.7039
Epoch 5/25
9896/9896 [==============================] - 520s 52ms/step - loss: 0.5325 - accuracy: 0.7164 - val_loss: 0.5273 - val_accuracy: 0.7134
Epoch 6/25
9896/9896 [==============================] - 521s 53ms/step - loss: 0.5214 - accuracy: 0.7256 - val_loss: 0.5684 - val_accuracy: 0.6732
Epoch 7/25
9896/9896 [==============================] - 519s 52ms/step - loss: 0.5109 - accuracy: 0.7327 - val_loss: 0

In [ ]:
model.save('name_model.h5')

In [ ]:
from google.colab import files

files.download('name_model.h5')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
with open('new_tokenizer.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)

files.download('new_tokenizer.pickle')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>